<a href="https://colab.research.google.com/github/KIMURAKAZUKI-ps/-/blob/main/MeasurementTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ドライブに接続

In [ ]:
# ドライブの認証
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import csv
import datetime
import os
import shutil
import glob
import sys
import gspread
from gspread.utils import a1_to_rowcol
from gspread.exceptions import WorksheetNotFound
from PIL import Image

# ユーザー設定セル
各変数の説明やツールの使い方詳細については[こちら](https://www.notion.so/present-square/SC-Readme-SmartCounterCycleTools-3bc149dfba8a4947b201043d100a5cb5?pvs=4#624db6f442fa48a691191510c9ee45bc)

In [ ]:
#@markdown
#@markdown ---
#@markdown  ### ◯ 共通設定
#@markdown 地点名フォルダの絶対パス
BASE_PATH = "/content/drive/Shareddrives/11_SmartCounter_\u6B69\u884C\u8005_\u30AF\u30E9\u30A4\u30A2\u30F3\u30C8\u5225_\u3042/17_NTT\u6771\u65E5\u672C\uFF08\u76DB\u5CA1\u5E02\u8A08\u6E2C\uFF09/2023\u5E7410\u6708_\u76DB\u5CA1\u5E02\u30AF\u30E9\u30D5\u30C8\u30D2\u3099\u30FC\u30EB\u30A4\u30D8\u3099\u30F3\u30C8\u4EA4\u901A\u91CF" #@param {type:"string"}
#@markdown 集計を行うタスクフォルダの名前（基本的にデフォルトのまま）
TASK_FOLDER_NAME = "task0" #@param ["task0", "task1", "task2", "task0-task1", "task1-task2", "task0-task2", "task0-task1-task2"] {allow-input: true}
#@markdown 計測対象について（vehicle or person）
TASK = "person"  #@param ["person", "vehicle"]
#@markdown スプレッドシートへ書き込むか否か(基本的に✅)
MATCHING = True  #@param {type:"boolean"}
#@markdown 書き込むスプレッドシートのID
MATHCING_SHEET_ID = "1xEwG92fh38E3Ck_M2yszZpVHPPdH2twL0yc8kkUNJNE" #@param {type:"string"}

#@markdown
#@markdown ---

#@markdown  ### ◯ デモ推論の集計の場合の設定
#@markdown デモ計測の集計を行う場合は✅
DEMO = True  #@param {type:"boolean"}

#@markdown
#@markdown ---

#@markdown ### ◯ 本計測の集計を行う場合の設定
#@markdown ５分毎とは別に集計したい時間単位を指定する
TIME_SPLIT = 10 #@param {type:"slider", min:10, max:60, step:5}
#@markdown 精度記録を行うか否か（本推論の場合は基本✅）
RECORD = False  #@param {type:"boolean"}
#@markdown 目視計測した列番号について（複数可。"/"で区切る。）
SEEING_ROW_LIST = "" #@param {type:"string"}
#@markdown 精度記録する際の地点情報（{クライアント名}_{地点名}で入力する。）
RECORD_NAME = "" #@param {type:"string"}

#@markdown
#@markdown ---

#@markdown ### ◯ Option
#@markdown すべての推論結果csvを結合したものを出力するか
OUTPUT_MERGED_CSV = False #@param {type:"boolean"}
#@markdown 自転車計測を行うか否か（personが検出対象にない場合は機能しない。）
BICYCLE = False #@param {type:"boolean"}
#@markdown バイク計測を行うか否か
MOTORCYCLE = False #@param {type:"boolean"}
#@markdown 分類修正を行うか否か
MODIFICATION = False #@param {type:"boolean"}

#@markdown ---

import sys
print('\033[36m'"logの集計を行います。"'\033[0m')
if TASK == "person":
  print("計測対象は歩行者です。")
else:
  print("計測対象は自動車です。")
if MODIFICATION == True:
  print('\033[36m'"画像分類の結果からAIの結果を修正します。"'\033[0m')

if MATCHING == True:
    # spreadsheetの認証
  from google.colab import auth
  import gspread
  from google.auth import default
  auth.authenticate_user()
  creds, _ = default()
  gc = gspread.authorize(creds)
  row_range_list = []
  for cell_range in SEEING_ROW_LIST.split("/"):
    row_range_list.append(cell_range)
  if MATHCING_SHEET_ID == "":
    print('\033[31m'"ERROR！：MATHCING_SHEET_IDを入力してください。"'\033[0m')
    sys.exit()
if DEMO == True:
  print("5分集計の結果を「デモ推論結果」シートに集計結果を書き込みます。")
  time_split_list = [5]
else:
  print(f"５分集計と{TIME_SPLIT}分の結果をそれぞれシートに書き込みます。")
  time_split_list = [5, TIME_SPLIT]

measure_task_list = TASK_FOLDER_NAME.split("-")


if RECORD == True :
  print('\033[36m'"突合した結果を精度管理票に記録します。必須情報を入力してください。"'\033[0m')
  if not len(RECORD_NAME.split("_")) == 2:
    print("RECORD_NAMEは「クライアント名_地点名」で入力してください。")
  print("以下の記録時に必要な設定セルをサムネイルと画角設定参考画像を元に入力してください。")
  print("")
  print("画角設定参考画像")
  if TASK == "person":
    img_path = "/content/drive/Shareddrives/11_SmartCounter_歩行者_学習用データ/00_学習用データ作成用ドキュメント・資料/画角設定参考画像.png"
  else:
    img_path = "/content/drive/Shareddrives/11_SmartCounter_自動車_学習用データ/00_学習用データ作成用ドキュメント/画角設定参考画像.png"
  Image.open(img_path).show() # 画像の表示
  print("")
  print("サムネイル")
  folder_path = glob.glob(os.path.join(BASE_PATH, "21_SmartCounter計測結果", "*.*"))[0]
  thumbneil_path = glob.glob(os.path.join(folder_path, TASK_FOLDER_NAME, "*.png"))
  if not thumbneil_path == []: # サムネイル画像が入ったフォルダの探索
    Image.open(thumbneil_path[0]).show() # 画像の表示
  else:
    print("サムネイル画像が見つかりません")

else:
  print('\033[1m'"画面上部の「ランタイム」から「以降のセルを実行」をクリックしてください"'\033[0m')



logの集計を行います。
計測対象は歩行者です。
5分集計の結果を「デモ推論結果」シートに集計結果を書き込みます。
画面上部の「ランタイム」から「以降のセルを実行」をクリックしてください


In [ ]:
#@markdown ### 以下RECORDにチェックを入れた場合のみ記入

#@markdown 画角設定参考画像を元にpersonであれば「画角_方向」vwhicleであれば「遠近感_方向」で記入
CAM_ANGLE = "" #@param {type:"string"}
if RECORD == True:
  if CAM_ANGLE == "":
    print("CAM_ANGLEを記入してください")
  else:
    if not len(CAM_ANGLE.split("_")) == 2:
      print("規定の入力方法で入力してください")
    else:
      print('\033[1m'"画面上部の「ランタイム」から「以降のセルを実行」をクリックしてください"'\033[0m')


# 実行セル

In [ ]:
# DEMO変数によってデモ推論と本推論を分岐
if DEMO == True: # デモ推論の場合

  # "21_SmartCounter計測結果"の中の"demo"フォルダ内のすべての"all_results*"という名前のフォルダやファイルのリストを取得
  date_list = glob.glob(os.path.join(BASE_PATH, "21_SmartCounter計測結果", "demo", "all_results*"))

  # "demo"フォルダが存在しない、または"all_results*"のフォルダやファイルが存在しない場合はエラーメッセージを表示して終了
  if date_list == []:
    print("21_SmartCounter計測結果にdemoフォルダが存在しません。")
    sys.exit()

else: # 本推論の場合
  # -----------推論のフォルダが、21_SmartCounter計測結果直下ではない場合は、必要なフォルダ名をall_resultsの前に追加する-------------------------------------------------
  date_list = glob.glob(os.path.join(BASE_PATH, "21_SmartCounter計測結果", "all_results*"))
  # --------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 取得した"all_results*"のリストを、フォルダ名の最後の日付でソート
sorted_date_list = sorted(date_list, key=lambda x: datetime.datetime.strptime(x.split("_")[-1], "%Y%m%d")) # フォルダ名の最後の日付に対してソートをかける


### 関数の定義

In [ ]:
# 自転車に乗っている人にフラグを立てる関数
def on_bicycle_flag(df):

  # "On_bicycle"カラムを初期化（全ての行を0に設定）
  df["On_bicycle"] = 0

  # dfから"bicycle"クラスのみを抽出して新しいデータフレームdf_bicycleを作成
  df_bicycle = df[(df["class"]=="bicycle")|(df["class"]=="motorcycle")].copy()
  df_bicycle = df_bicycle.reset_index()

  # 自転車のデータと、そのデータの±1秒以内に存在する他のデータを紐づけるためのロジック
  lost = []    # # 紐づけができなかった自転車データのインデックスを保存するリスト
  for i in range(len(df_bicycle)):

    # 自転車データの±1秒以内、かつ同じ方向に動いている、かつ自転車ではないデータを抽出
    df_ = df[(df_bicycle["time"][i]-1<df["time"]) & (df["time"]<df_bicycle["time"][i]+1) & (df["go_to"]==df_bicycle["go_to"][i])& (df["class"]!=df_bicycle["class"][i])]

    # 上記条件に合致するデータが存在する場合
    if len(df_)>0:

      # 最も自転車データの時間に近いレコードのインデックスを取得
      person_idx = df_['time'].sub(df_bicycle["time"][i]).abs().idxmin()

      # その人物データに"On_bicycle"フラグを立てる
      df.loc[person_idx, 'On_bicycle'] = 1

      # 人の座標を自転車のものに合わせる
      df.loc[person_idx, 'in_box'] = in_box
      df.loc[person_idx, 'out_box'] = out_box
    else:
      matching_rows = df[(df["class"] == df_bicycle["class"][i]) & (df["time"] == df_bicycle["time"][i])]
      df.loc[matching_rows.index, 'lost_person'] = 1
  return df

# 集計したdataframeを作成する
def totalize_df(df, start_date, end_date, time_split, cls):

  # 'come_from', 'go_to', 'class', 'Minute' の組み合わせごとにデータを集計
  result = df.groupby(['come_from', 'go_to', 'class', 'Minute']).size().unstack(fill_value=0)

  # 期待するマルチインデックスを作成
  multi_index = pd.MultiIndex.from_product([come_from, go_to, cls], names=['come_from', 'go_to', 'class'])

  # 期待するマルチインデックスに基づいてresultのインデックスを再編成し、該当しないインデックスは0で埋める
  result = result.reindex(index=multi_index, fill_value=0).T

  # 不要なカラムを削除
  result = result.drop(columns=columns_to_drop)

  # 指定された開始日時と終了日時の間の時間範囲を生成
  hours_range = pd.date_range(start=start_date+'000000', end=end_date+'235900', freq=f'{time_split}T').strftime('%Y-%m-%d %H:%M:%S')

  # 上記時間範囲に基づいてresultのインデックスを再編成し、該当しないインデックスは0で埋める
  result = result.reindex(hours_range, fill_value=0)

  # インデックス名を"Time"に設定
  result.index.name = 'Time'
  return result


# 集計した結果を指定したスプレッドシートに書き込む関数
def write_df_2_gspread(workbook, sheet_name, df, cell, matching_copy=True):
  print(f"{sheet_name}へ書き込みを行います。")
  try:
    # シートが存在するか確認
    worksheet = workbook.worksheet(sheet_name)

  except WorksheetNotFound:
    # シートが存在しなかった場合、必要なシートをコピーし、リネームする
    # AI記入用のシートコピー
    source_worksheet = workbook.worksheet("本推論結果")
    source_worksheet.duplicate(new_sheet_name=sheet_name)
    worksheet = workbook.worksheet(sheet_name)

    # 突合用のシートもコピーする場合の処理
    if matching_copy == True:
      # 突合用シートのコピー
      source_worksheet1 = workbook.worksheet("【突合結果】本推論と抜粋(COUNTIFS集計)")
      source_worksheet.duplicate
      source_worksheet2 = workbook.worksheet("【突合結果】本推論と抜粋(%)(COUNTIFS集計)")
      source_worksheet.duplicate(new_sheet_name=f"【突合結果】{sheet_name}と抜粋(%)※関数未修正")
  finally:

    # データフレイムの情報をリスト化し、シートに書き込む
    df = df.reset_index(drop=True)
    df = df.droplevel(0, axis=1)      # マルチインデックスの最上位レベルを削除
    df = df.astype(int)
    values_list = df.values.tolist()

    # シートにデータを書き込む
    worksheet.update(cell, values_list)
    print("finished")



### 基本情報の取得

In [ ]:

if TASK == "person":
  classes = ["M05", "M15", "M25", "M35", "M45", "M55", "M65", "M75", "F05", "F15", "F25", "F35", "F45", "F55", "F65", "F75"]
  # classes = ["person"]
elif TASK == "vehicle":
  classes = ["normal", "smalltruck", "truck", "bus"]

if MATCHING == True:
  # 入力するセルのカラム範囲をタスクに応じて指定
  if TASK == "vehicle":
    start_column = "AD"
    end_column = "BY"
  else:
    start_column = 'X'
    end_column = 'HG'

  # 突合用のスプレッドシートのワークブックを取得
  matching_workbook = gc.open_by_key(MATHCING_SHEET_ID)
  print("↓記録するスプレッドシート情報↓")
  print(matching_workbook)

  # 記録するスプレッドシートのシート名を設定
  seeing_sheetname = "抜粋目視計測"

  # DEMOフラグに応じてAI、自転車、バイクの推論結果のシート名を指定
  if DEMO == True:
    AI_sheet_names = ["デモ推論結果"]
    bic_sheet_names = ["デモ推論結果_bicycle"]
    moto_sheet_names = ["デモ推論結果_motorcycle"]
  else:
    AI_sheet_names = ["本推論結果_5", f"本推論結果_{TIME_SPLIT}"]
    bic_sheet_names = ["本推論結果_bicycle_5", f"本推論結果_bicycle_{TIME_SPLIT}"]
    moto_sheet_names = ["本推論結果_motorcycle_5", f"本推論結果_motorcycle_{TIME_SPLIT}"]


↓記録するスプレッドシート情報↓
<Spreadsheet '2023-10-07(土曜)-盛岡市_person_計測結果突合シート' id:1xEwG92fh38E3Ck_M2yszZpVHPPdH2twL0yc8kkUNJNE>


### log修正ツール

In [ ]:
# 分類修正の処理が有効の場合
if MODIFICATION == True:

  # 各タスクと日付の組み合わせで処理を行う
  for task_name in measure_task_list:
    for date_path in sorted_date_list:

      # 指定したタスクのCSVファイルリストを取得
      file_list = sorted(glob.glob(os.path.join(date_path, f"*{task_name}.csv")))
      for file_path in file_list:
        num = 0

        # 元のCSVデータを読み込む
        source_data = pd.read_csv(file_path)

        # ファイル名から目的のフォルダ名を生成
        file_name = os.path.basename(file_path)
        target_folder_name = "_".join(file_name.split("_")[:-1])

        inf_pwd_path = os.path.dirname(os.path.dirname(file_path)) # 推論結果フォルダの親フォルダのパス
        mod_path = os.path.join(inf_pwd_path, target_folder_name, task_name, "mod") # 分類修正を行った画像が格納されているフォルダのパス

        # 元のcsvを第２引数へ移動
        move_to_path = os.path.join(inf_pwd_path, "分類修正前_all_results")
        if not os.path.exists(move_to_path):
          os.makedirs(move_to_path)
        if not os.path.join(move_to_path, file_name):
          shutil.move(file_path, move_to_path)

        # "objectID"をインデックスとして使用する
        data = source_data.set_index("objectID")

        # 分類修正後の画像フォルダから、各クラスごとの画像を読み込み、正解ラベルを付与
        for cls in classes:
          cls_path = os.path.join(mod_path, cls)
          for img_path in sorted(glob.glob(os.path.join(cls_path, "*"))):
            img_name = os.path.basename(img_path)
            img_name_split = img_name.split("_")

             # 画像名からobjectIDを抽出
            try:
              object_id = int(img_name_split[-1].split(".")[0]) # 画像名からidxを取得
            except ValueError:
              object_id = int(img_name_split[-1].split("(")[0]) # 画像名からidxを取得

            ######## 同一のobjectIDが存在するケースが確認されたのでこちらで運用中

            # 画像名に正解ラベルを付与し、データフレームを更新
            data.at[object_id, "class"] = cls

            # 画像のファイル名を正しいクラス名で上書き
            img_pwd_path = os.path.dirname(img_path)
            img_name_split[1] = cls
            new_name = "_".join(img_name_split)
            shutil.move(img_path, os.path.join(img_pwd_path, new_name))
            ############
            # if not data.at[object_id, "class"] == cls:
            #   data.at[object_id, "class"] = cls # dataframeのclassを修正
            #   # 画像名を正しいクラスに上書き
            #   img_pwd_path = os.path.dirname(img_path)
            #   img_name_split[1] = cls
            #   new_name = "_".join(img_name_split)
            #   print(img_name, new_name)
            #   # shutil.move(img_path, os.path.join(img_pwd_path, new_name))

        # 分類修正後のデータをCSVとして出力
        mod_name = file_name # 分類修正前と同じ名前・フォルダに出力
        mod_csv_path = os.path.join(os.path.dirname(file_path), mod_name)
        data.to_csv(mod_csv_path)

### log集計ツール & SSへの記入

In [ ]:
# time_split_listの各要素ごとに集計処理を実施
for i in range(len(time_split_list)):
  time_split = time_split_list[i]

  # 各タスクごとのlogを集計
  for j in range(len(measure_task_list)):
    df_ = pd.DataFrame() # 空のdataframeの作成
    task_name = measure_task_list[j]

    # 各日付のデータを集計
    for date_path in sorted_date_list:
      if date_path == sorted_date_list[0]: # 日付順にソートしたリストの先頭なら開始日
        start_date = os.path.basename(date_path).split("_")[-1]
      if date_path == sorted_date_list[-1]: # 日付順にソートしたリストの最後なら終了日
        end_date = os.path.basename(date_path).split("_")[-1]

      # タスク名に一致するCSVファイルを取得
      file_list = sorted(glob.glob(os.path.join(date_path, f"*{task_name}.csv")))

      total_sec = 0
      for file_path in file_list:
        file_name = os.path.basename(file_path)
        print(file_name)

        # ファイル名から日時情報を取得
        file_info = file_name.split("_")
        ymd1 = file_info[2].split("-")[0]
        hms1 = file_info[2].split("-")[1]
        sec1 = (int(hms1[:2]) * 3600) + (int(hms1[2:4]) * 60) + int(hms1[4:])
        ymd2 = file_info[3].split("-")[0]
        hms2 = file_info[3].split("-")[1]
        if ymd1 == ymd2:
          print(True, ymd1, ymd2)
          sec2 = (int(hms2[:2]) * 3600) + (int(hms2[2:4]) * 60) + int(hms2[4:])
        else: # 日をまたいでいる場合は24時間（86400秒）足す
          print(False, ymd1, ymd2)
          sec2 = (int(hms2[:2]) * 3600) + (int(hms2[2:4]) * 60) + int(hms2[4:]) + 86400
        video_time = int(file_info[4].split(".")[0])

        # CSVファイルを読み込み、時間情報を加工
        df = pd.read_csv(file_path)
        base_time = pd.to_datetime(ymd1+hms1, format='%Y%m%d%H%M%S')
        df["origin_time"] = df["time"]
        df["time"] = df["time"] * ((sec2-sec1) / video_time) # csvのtimeを時刻のスケールに合わせる
        df["time_delta"] = base_time + pd.to_timedelta(df['time'], unit='s') # timeをtime_delta型へ変換
        df['Minute'] = df['time_delta'].dt.floor(f'{time_split}T') # 各行がどの時間帯のものかの行を追加
        total_sec += video_time
        df["time"] = df["time"] + total_sec
        df_ = pd.concat([df_, df], ignore_index=True) # 各csvを結合

    # 結果をCSVとして出力
    df_.to_csv(os.path.join(date_path,f"{task_name}-{time_split}-merged.csv"))
    come_from = ['up', 'down', 'left', 'right']
    go_to = ['up', 'down', 'left', 'right']
    columns_to_drop = [("up", "up"), ("down", "down"), ("left", "left"), ("right", "right")]

    # 集計結果を保存するディレクトリを作成
    output_path = os.path.join(os.path.dirname(date_path), "00_aggregation_results")
    if not os.path.exists(output_path):
      os.makedirs(output_path)

    # 自転車およびバイクのデータを分離し、結果を集計
    if BICYCLE == True or MOTORCYCLE == True:
      # 自転車のデータ集計
      if BICYCLE == True:
        b_totalize_df = on_bicycle_flag(df_) # bicycle計測を行う場合は「on_bicycle」カラムを作成する
        main_df = b_totalize_df.groupby("On_bicycle").get_group(0) # on_bicycleにフラグが立っていない = 自転車に乗っていない人
        try:
          bic_df = b_totalize_df.groupby("On_bicycle").get_group(1) # on_bicycleにフラグが立っている = 自転車に乗っている人
        except KeyError:
          print("自転車の交通がありませんでした。")
          bic_df = pd.DataFrame(columns=main_df.columns)
        bicycle_result = totalize_df(bic_df, start_date, end_date, time_split, classes)
        bicycle_result.to_csv(os.path.join(output_path, f"{time_split}min_{task_name}_bicycle_result.csv"))

       # バイクのデータ集計
      if MOTORCYCLE == True:
        b_totalize_df = df_
        moto_df = b_totalize_df[b_totalize_df["class"] == "motorcycle"] # clsがmotorcycleである行を抽出
        motorcycle_result = totalize_df(moto_df, start_date, end_date, time_split, ["motorcycle"])
        motorcycle_result.to_csv(os.path.join(output_path, f"{time_split}min_{task_name}_motorcycle_result.csv"))
        main_df = b_totalize_df[b_totalize_df["class"] != "motorcycle"]

    else:
      main_df = df_

    # 主要なデータを集計し、CSVとして出力
    main_result = totalize_df(main_df, start_date, end_date, time_split, classes) # 残ったdfはTASKに該当するクラスのみになっている
    main_result.to_csv(os.path.join(output_path, f"{time_split}min_{task_name}_{TASK}_result.csv"))

    if j == 0:
      if BICYCLE == True:
        total_bicycle_result = bicycle_result
      if MOTORCYCLE == True:
        total_motorcycle_result = motorcycle_result
      total_main_result = main_result
    else:
      # 現在のタスク名を表示
      print(task_name)

      # 自転車のデータが存在する場合、集計結果に加算
      if BICYCLE == True:
        total_bicycle_result += bicycle_result
        # すべてのタスクが完了した場合、CSVとして出力
        if len(measure_task_list) == j+1:
          total_bicycle_result.to_csv(os.path.join(output_path, f"{time_split}min_{TASK_FOLDER_NAME}_bicycle_result.csv"))
      if MOTORCYCLE == True:
        total_motorcycle_result += motorcycle_result
        # すべてのタスクが完了した場合、CSVとして出力
        if len(measure_task_list) == j+1:
          total_motorcycle_result.to_csv(os.path.join(output_path, f"{time_split}min_{TASK_FOLDER_NAME}_motorcycle_result.csv"))
      # 主要なデータの集計結果に加算
      total_main_result += main_result
      # すべてのタスクが完了した場合、CSVとして出力
      if len(measure_task_list) == j+1:
        total_main_result.to_csv(os.path.join(output_path, f"{time_split}min_{TASK_FOLDER_NAME}_{TASK}_result.csv"))

  # スプレッドシートへの記入
  if MATCHING == True:

    # スプレッドシートの各シート名を取得
    AI_sheet_name = AI_sheet_names[i]
    print(AI_sheet_name)
    bic_sheet_name = bic_sheet_names[i]
    moto_sheet_name = moto_sheet_names[i]

    # 記入を開始するセルを指定
    write_cell = f"{start_column}5"

    # 主要なデータをスプレッドシートに記入
    write_df_2_gspread(matching_workbook, AI_sheet_name, total_main_result, write_cell, time_split)

    # 自転車のデータが存在する場合、それをスプレッドシートに記入
    if BICYCLE == True:
      write_df_2_gspread(matching_workbook, bic_sheet_name, total_bicycle_result, write_cell, time_split)

    # バイクのデータが存在する場合、それをスプレッドシートに記入
    if MOTORCYCLE == True:
      write_df_2_gspread(matching_workbook, moto_sheet_name, total_motorcycle_result, "Z5", time_split)

NTT_盛岡202310_20231007-100000_20231007-110000_3600_.mp4_task0.csv
True 20231007 20231007
NTT_盛岡202310_20231007-110000_20231007-120000_3600_.mp4_task0.csv
True 20231007 20231007
NTT_盛岡202310_20231007-120000_20231007-130000_3596_.mp4_task0.csv
True 20231007 20231007
NTT_盛岡202310_20231007-150000_20231007-160000_3600_.mp4_task0.csv
True 20231007 20231007
デモ推論結果
デモ推論結果へ書き込みを行います。


<ipython-input-26-1434722bd900>:45: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = result.drop(columns=columns_to_drop)


finished


# 精度記録

In [ ]:
# 精度をスプレッドシートに記録するかどうかの判定と、デモモード・記録モードの判定
if MATCHING == True and DEMO == False and RECORD == True:

  # ユーザーに確認を求める
  confirm = input(f"{RECORD_NAME}という記録名で記録しますがよろしいですか？[Y/n]")
  if confirm == ("y" or "Y"):
    #記録用スプレッドシート情報の取得
    if TASK == "person":
      record_sheet_id = "1Ezc4oi_33lIXjcsQSmipjF_qXzCllxXz_95ixN7hgqk"
    elif TASK == "vehicle":
      record_sheet_id = "1m7XN-PpE_SMtnlSLhNxbXIVVgSDIdG7ABTOsWBwBNLw"
    rec_workbook = gc.open_by_key(record_sheet_id)
    rec_worksheet = rec_workbook.worksheet("記入シート") # 精度を記録するシートを指定

    # 人計測結果の取得
    read_seeing_worksheet = matching_workbook.worksheet(seeing_sheetname)
    read_AI_worksheet = matching_workbook.worksheet("本推論結果_5")
    write_list = []
    col_list = classes * 12
    for k in range(len(row_range_list)): # ユーザー設定セルにて、抜粋目視をしたセルの情報を取り出す
      row = row_range_list[k]
      value_range = f'{start_column}{row}:{end_column}{row}'
      try:
        human_match_df = pd.DataFrame(read_seeing_worksheet.batch_get([value_range])[0])
      except KeyError:
        print("\033[36m抜粋目視計測シートの指定した範囲に値が入力されていない可能性があります。ご確認お願いします。\033[0m")
        sys.exit()
      human_match_df = human_match_df.replace('', 0) # 空欄があった場合は0で埋める
      human_match_df = human_match_df.astype(int) # 要素をint型にする
      try:
        human_match_df.columns = col_list
      except ValueError:
        print(f"\033[31m ERROR：抜粋目視計測シートの{end_column}{row}に０を入力してもう一度実行してください。\033[0m")
        sys.exit()

      AI_match_df = pd.DataFrame(read_AI_worksheet.batch_get([value_range])[0]) # 取り出した行数に該当する行を取り出す
      AI_match_df = AI_match_df.replace('', 0) # 空欄があった場合は0で埋める
      AI_match_df = AI_match_df.astype(int) # 要素をint型にする
      AI_match_df.columns = col_list



      # 記録情報の取得
      # prob_df = round(AI_match_df / human_match_df, 3)
      prob_df = round(pd.DataFrame(np.where(human_match_df != 0, AI_match_df / human_match_df, AI_match_df)), 3) # 目視の結果が0の場合はAIの値をそのまま出力
      prob_df.columns = col_list         # DataFrameのカラムを設定
      prob_df.fillna(0, inplace=True)    # NAの値を0で埋める
      # 各クラスの精度の平均
      mean_prob_df = prob_df.groupby(prob_df.columns, axis=1).mean().mean().reindex(classes)
      mean_prob_df.fillna(0, inplace=True)

      # タスクが「人」の場合、性別ごとの平均精度を計算
      if TASK == "person":
        m_prob = round(prob_df.groupby(lambda x: x.startswith('M'), axis=1).mean()[True], 3)
        w_prob = round(prob_df.groupby(lambda x: x.startswith('F'), axis=1).mean()[True], 3)

      # AIの計測結果と目視の計測結果の合計値を取得
      AI_sum = AI_match_df.sum().sum()
      human_sum = human_match_df.sum().sum()

      # 合計値の比較結果に基づいて精度を計算
      if human_sum == 0:
        prob_dir = AI_sum
      elif AI_sum == 0:
        prob_dir = 0
      else:
        prob_dir = round(AI_sum / human_sum, 3)

      # 光量の取得
      time_cell = f"F{row}"
      target_time = datetime.datetime.strptime(read_AI_worksheet.batch_get([time_cell])[0][0][0], "%H:%M:%S").time() # 抜粋目視した時間（F列）を取得し、datetime型に変換
      start_time = datetime.datetime.strptime("06:00:00", "%H:%M:%S").time() # 該当の行の開始時刻を取得
      end_time = datetime.datetime.strptime("18:00:00", "%H:%M:%S").time() # 該当の行の終了時刻を取得
      if start_time <= target_time <= end_time: # 6時から18時の場合は明とする。
          target_brightness = "明"
      else:
          target_brightness = "暗"

      rec_list = []
      today = datetime.date.today().strftime('%Y/%m/%d')
      rec_list.append(today) # 記録日
      rec_list.append(RECORD_NAME) # 場所名
      rec_list.append(target_brightness) # 光量
      rec_list.append(CAM_ANGLE.split("_")[0]) # 画角情報1
      rec_list.append(CAM_ANGLE.split("_")[1]) # 画角情報2
      rec_list.append('計測案件')
      rec_list.append(prob_dir)

      # タスクが「人」の場合、性別ごとの精度情報をリストに追加
      if TASK == "person":
        rec_list.append(m_prob[0])
        rec_list.append(w_prob[0])

      # 各クラスごとの精度情報をリストに追加
      for cls_acc in mean_prob_df:
        rec_list.append(round(cls_acc, 3))

      # スプレッドシートの指定したワークシートにリストの情報を新しい行として追加
      rec_worksheet.append_row(rec_list)
